In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from plotting.constants import PROJECT_NAME_BASE_BO
from plotting.utils import get_runs, get_metrics_and_colors

In [3]:
metric = "fn_max"
lengthscale = 2.0 # 3.0

In [4]:
runs = get_runs(PROJECT_NAME_BASE_BO + str(lengthscale), mode="bo")

# Split runs based on seeds
runs_split = {}
for run in runs:
    seed_key = f"Seed {run.run.config['seed']}"
    if seed_key not in runs_split:
        runs_split[seed_key] = []
    runs_split[seed_key].append(run)

In [5]:
# Get percentage improvement over random search for each seed + optimizer combo
percent_improvement = {}
for seed_key, runs in runs_split.items():
    metrics_dict, _ = get_metrics_and_colors(runs, metric)

    random_search_baseline = metrics_dict["Random Search"][0].metric_data[-1]
    print(f"seed_key: {seed_key}, random_search_baseline: {random_search_baseline}")
    for key, data in metrics_dict.items():
        if len(data) != 1:
            raise ValueError(
                f"Expected only one run for {key}, but got {len(data)}"
            )
        
        if key == "Random Search":
            continue
        # Calculate percent improvement
        if key not in percent_improvement:
            percent_improvement[key] = []
        print(f"key: {key}, data: {data[0].metric_data[-1]}")
        percent_improvement[key].append(
            (data[0].metric_data[-1] - random_search_baseline) /
            random_search_baseline * 100
        )

print(percent_improvement)

seed_key: Seed 0, random_search_baseline: 2.046401023864746
key: PCG, data: 2.0522234439849854
key: \texttt{ADASAP}, data: 2.453432083129883
key: \texttt{ADASAP-I}, data: 2.350476026535034
key: SDD-1, data: 2.326610803604126
key: SDD-10, data: 2.046401023864746
seed_key: Seed 1, random_search_baseline: 2.317646026611328
key: PCG, data: 2.317646026611328
key: \texttt{ADASAP}, data: 2.4974818229675293
key: \texttt{ADASAP-I}, data: 2.3285958766937256
key: SDD-1, data: 2.359241008758545
key: SDD-10, data: 2.317646026611328
seed_key: Seed 2, random_search_baseline: 1.8594634532928467
key: PCG, data: 1.8594634532928467
key: \texttt{ADASAP}, data: 1.9792566299438477
key: \texttt{ADASAP-I}, data: 1.9380643367767334
key: SDD-1, data: 1.9413827657699585
key: SDD-10, data: 1.8594634532928467
seed_key: Seed 3, random_search_baseline: 4.2918548583984375
key: PCG, data: 4.2918548583984375
key: \texttt{ADASAP}, data: 4.593835353851318
key: \texttt{ADASAP-I}, data: 4.549937725067139
key: SDD-1, data: 

In [6]:
# Average percent improvement over all optimizers
avg_percent_improvement = {}
for key, values in percent_improvement.items():
    avg_percent_improvement[key] = np.mean(values)

In [7]:
print(avg_percent_improvement)

{'PCG': np.float64(0.12852610231828998), '\\texttt{ADASAP}': np.float64(10.424931481163016), '\\texttt{ADASAP-I}': np.float64(7.037693759716153), 'SDD-1': np.float64(6.496394569052091), 'SDD-10': np.float64(0.0)}
